In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# CDF Response Time Analysis Example 2
# Demonstrates class switching in a closed network with 3 classes

model = Network('model')

# Create closed queueing network with Delay and Queue nodes
node = [None] * 2  # using 0-based indexing
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue2', SchedStrategy.PS)

# Three closed classes - only Class1 has initial population
jobclass = [None] * 3  # using 0-based indexing
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0], 0)
jobclass[2] = ClosedClass(model, 'Class3', 0, node[0], 0)

# Class1 doesn't complete (stays in the system for class switching)
jobclass[0].completes = False

# Service processes at Delay node
node[0].set_service(jobclass[0], Exp(1/1))  # rate = 1
node[0].set_service(jobclass[1], Exp(1/1))  # rate = 1
node[0].set_service(jobclass[2], Exp(1/1))  # rate = 1

# Service processes at Queue node
node[1].set_service(jobclass[0], Exp(1/1))      # rate = 1
node[1].set_service(jobclass[1], Erlang(1/2, 2)) # Erlang with rate=1/2, order=2
node[1].set_service(jobclass[2], Exp(1/0.01))   # rate = 100

# Complex routing matrix with class switching
P = model.init_routing_matrix()

# Class 1 to Class 1: Delay->Queue2
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)

# Class 1 to Class 2: Queue2->Delay (class switch)
P.set(jobclass[0], jobclass[1], node[1], node[0], 1.0)

# Class 2 to Class 1: Delay->Queue2 (class switch back)
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

# Class 2 to Class 2: Delay->Queue2
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# Class 3 circulates within itself
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [ ]:
# NOTE: The JAR test shows that JMT getCdfRespT is not implemented for multi-class models
# So we only implement the Fluid solver scenario that works

# Solve with Fluid solver - aligned with JAR test scenario
options = FLD.default_options()
options.method = 'statedep'  # Aligned with JAR test: method="statedep"
options.iter_max = 100  # Aligned with JAR test: iter_max=100

solver = FLD(model, options)
AvgRespT = solver.avg_respt()
print('AvgRespT =')
print(AvgRespT)

# Get CDF of response times
FC = solver.cdf_respt()

# Calculate statistics from CDF
AvgRespTfromCDF = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
PowerMoment2_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
Variance_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
SqCoeffOfVariationRespTfromCDF = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(model.get_number_of_stations()):
    for c in range(model.get_number_of_classes()):
        if FC[i][c] is not None and len(FC[i][c]) > 1:
            # Calculate mean from CDF
            diffs = np.diff(FC[i][c][:, 0])
            values = FC[i][c][1:, 1]
            AvgRespTfromCDF[i, c] = np.sum(diffs * values)
            
            # Calculate second moment and variance
            PowerMoment2_R[i, c] = np.sum(diffs * (values ** 2))
            Variance_R[i, c] = PowerMoment2_R[i, c] - AvgRespTfromCDF[i, c] ** 2
            
            # Avoid division by zero (like MATLAB's implicit handling)
            if AvgRespTfromCDF[i, c] != 0:
                SqCoeffOfVariationRespTfromCDF[i, c] = Variance_R[i, c] / (AvgRespTfromCDF[i, c] ** 2)
            else:
                SqCoeffOfVariationRespTfromCDF[i, c] = np.nan

print('AvgRespTfromCDF =')
print(AvgRespTfromCDF)
print('SqCoeffOfVariationRespTfromCDF =')
print(SqCoeffOfVariationRespTfromCDF)

# Commented out JMT solver scenario - JAR test shows it's disabled for multi-class CDF models
# # Solve with JMT solver - aligned with JAR test scenario (disabled in JAR)
# # solver = JMT(model, seed=23000, samples=10000)
# # FC_jmt = solver.cdf_respt()
# # print("JMT solver CDF analysis would be here, but it's not implemented for multi-class models")